COMP 215 - LAB 2 Records (NEO)
----------------
#### Name: Serena Harrington
#### Date: 12/1/26

This lab exercise is mostly a review of strings, tuples, lists, dictionaries, and functions.

**Building on new concepts from lab 1**:
  * `datetime.date` objects represent a calendar date
  * *list comprehension* provides a compact way to represent map and filter algorithms

**New Python Concepts**:
  * *f-string* simplifies string formatting operations

As usual, the first code cell simply imports all the modules we'll be using...

In [2]:
import json, requests
from datetime import date, datetime
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/)
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

You should register for your own API key, (but may use the DEMO_KEY to get started).

First, a short reminder about python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [3]:
today = date.today()   # get a date object representing today's date
print(today, type(today))
# f"..."  is a format string - notice how the variable `today` is formatted into the string
print(f'Today is: {today}')

2026-01-15 <class 'datetime.date'>
Today is: 2026-01-15


### Make a query

Let's get some data from the NEO database...
Here's a query that gets the observation "feed" for today.
(Note: I hard-coded the date below to lock down the data for the lab - ideally used `today()` so the notebook is always up-to-date.)

In [4]:
NASA_NEO_FEED_API_URL = "https://api.nasa.gov/neo/rest/v1/feed"
API_KEY = 'DEMO_KEY'  # substitute your API key here

start_date = '2023-01-09'  #  Future enhancement:  str(date.today())   # Today's date as a string!

feed_params = dict(
    start_date=start_date,
    end_date=start_date,
    api_key=API_KEY,
)

response = requests.request("GET", NASA_NEO_FEED_API_URL, params=feed_params)
response.raise_for_status()  # raise an error if request is not successful

feed_data = json.loads(response.text)  # recall json.loads from lab 1

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = feed_data['element_count']
neos = feed_data['near_earth_objects'][start_date]
print(f'{n_results} Near Earth Objects found for {start_date}')
pprint(neos[:2])

11 Near Earth Objects found for 2023-01-09
[{'absolute_magnitude_h': 19.62,
  'close_approach_data': [{'close_approach_date': '2023-01-09',
                           'close_approach_date_full': '2023-Jan-09 13:59',
                           'epoch_date_close_approach': 1673272740000,
                           'miss_distance': {'astronomical': '0.0725378623',
                                             'kilometers': '10851509.694433301',
                                             'lunar': '28.2172284347',
                                             'miles': '6742815.4556135138'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '32387.3479203461',
                                                 'kilometers_per_second': '8.9964855334',
                                                 'miles_per_hour': '20124.2384897284'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 2322.8706681664,
     

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [5]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified on {start_date}.')

2 potentially hazardous asteroids identified on 2023-01-09.


## Exercise 1

In the code cell below, **re-write the accumulator loop above** as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [6]:
# Ex. 1 your code here
hazards = [hazard for hazard in neos if hazard['is_potentially_hazardous_asteroid'] is True]
f'{len(hazards)} potentially hazardous asteroids identified on {start_date}.'

'2 potentially hazardous asteroids identified on 2023-01-09.'

## Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.

We can use this `id` to fetch complete orbital and close approach data for the NEO.

For example, this query fetches the complete data set for the first hazardous asteroid...


In [7]:
NASA_NEO_API_URL = "https://api.nasa.gov/neo/rest/v1/neo/{id}"
neo_params = dict(
    api_key=API_KEY,
)
hazard_id = hazards[0]['id']

response = requests.request("GET", NASA_NEO_API_URL.format(id=hazard_id), params=neo_params)
response.raise_for_status()  # raise an error if request is not successful

neo_data = json.loads(response.text)

print(f'NEO {neo_data["name"]} with {len(neo_data["close_approach_data"])} approach records.  {"Hazardous." if neo_data["is_potentially_hazardous_asteroid"] else ""}')
print(f'Close approach records:')
close_approach_data = neo_data['close_approach_data']
pprint(close_approach_data[:2])

NEO 226554 (2003 WR21) with 130 approach records.  Hazardous.
Close approach records:
[{'close_approach_date': '1900-10-30',
  'close_approach_date_full': '1900-Oct-30 21:15',
  'epoch_date_close_approach': -2182819500000,
  'miss_distance': {'astronomical': '0.4183488933',
                    'kilometers': '62584103.354537271',
                    'lunar': '162.7377194937',
                    'miles': '38887958.5659094998'},
  'orbiting_body': 'Earth',
  'relative_velocity': {'kilometers_per_hour': '30027.9671888418',
                        'kilometers_per_second': '8.3411019969',
                        'miles_per_hour': '18658.2110568667'}},
 {'close_approach_date': '1901-06-26',
  'close_approach_date_full': '1901-Jun-26 20:27',
  'epoch_date_close_approach': -2162172780000,
  'miss_distance': {'astronomical': '0.0308794402',
                    'kilometers': '4619498.480712374',
                    'lunar': '12.0121022378',
                    'miles': '2870423.2526198012'},
  '

Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".

## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [20]:
# Ex. 2 your code here

dates_miss_km = [(approach['close_approach_date'], approach['miss_distance']['kilometers']) for approach in close_approach_data if approach['orbiting_body'] == 'Earth']

tuple_with_lowest_dist = dates_miss_km[0]
closest_miss_km = dates_miss_km[0][1]

for approach in dates_miss_km:
  if approach[1] < closest_miss_km:
    closest_miss_km = approach[1]
    tuple_with_lowest_dist = approach

print(tuple_with_lowest_dist)



('2049-01-09', '10799721.724279257')


## Challenge - Take your skills to the next level...
## Exercise 3

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, when it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [9]:
# Ex. 3 (challenge) your code here